In [ ]:
# Needs hdf5 to run
# Instead of hf datasets lib alternatively use torchgeo
# As of 02.11.24 failed to reach geo API, could attempt later
%pip install --upgrade torch;
%pip install --upgrade torchvision;
%pip install --upgrade datasets;
%pip install --upgrade h5py;
%pip install --upgrade kornia

In [2]:
# Copyright 2020 The HuggingFace Datasets Authors and the current dataset script contributor.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import os
import sys
import torch
import datasets
import PIL
# from torchgeo.datasets import CaBuAr
from torch.utils.data import DataLoader
from torchvision.transforms import v2 as transforms
import torch.nn as nn
from torchvision.transforms.v2 import functional as F
import h5py
import numpy as np
import matplotlib.pyplot as plt
from kornia import filters
import kornia.augmentation as K


# Find for instance the citation on arxiv or on the dataset repo/website
_CITATION = """\
@article{cabuar,
  title={Ca{B}u{A}r: California {B}urned {A}reas dataset for delineation},
  author={Rege Cambrin, Daniele and Colomba, Luca and Garza, Paolo},
  journal={IEEE Geoscience and Remote Sensing Magazine},
  doi={10.1109/MGRS.2023.3292467},
  year={2023}
}
"""

# You can copy an official description
_DESCRIPTION = """\
CaBuAr dataset contains images from Sentinel-2 satellites taken before and after a wildfire.
The ground truth masks are provided by the California Department of Forestry and Fire Protection and they are mapped on the images.
"""

_HOMEPAGE = "https://huggingface.co/datasets/DarthReca/california_burned_areas"

_LICENSE = "OPENRAIL"

# Define the root directory for the dataset
# Change manually


_URLS = {'root': os.curdir,'cache_dir':"raw"}
_BATCH_SIZE = 16
_NUM_WORKERS = 2
_TRAIN_SPLIT = 1
_VALID_SPLIT = 2
_TEST_SPLIT = 3


Тут можна задати спліти 0-4, що дає вчитися і перевіряти метрики на незалежних вибірках



In [3]:
# Set the Hugging Face datasets cache directory
os.environ['HF_DATASETS_CACHE'] = os.path.join(_URLS['root'],_URLS['cache_dir'])
sys.path.append(os.path.abspath('..'))

# You can verify by printing (optional)
print(f"HF_DATASETS_CACHE set to: {os.getenv('HF_DATASETS_CACHE')}")

HF_DATASETS_CACHE set to: ./raw


In [4]:
# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def to_device(X, y):
    return X.to(device), y.to(device, dtype=torch.int64)

In [5]:
# Load the CaBuAr dataset
# Possible splits: 1-5, or chabud
# For docs check out
# https://huggingface.co/datasets/DarthReca/california_burned_areas
# https://torchgeo.readthedocs.io/en/stable/api/datasets.html#cabuar
train_data = datasets.load_dataset('DarthReca/california_burned_areas', name='pre-post-fire', split=f'{_TRAIN_SPLIT}',trust_remote_code=True)
valid_data = datasets.load_dataset('DarthReca/california_burned_areas', name='pre-post-fire', split=f'{_VALID_SPLIT}',trust_remote_code=True)
test_data = datasets.load_dataset('DarthReca/california_burned_areas', name='pre-post-fire', split=f'{_TEST_SPLIT}',trust_remote_code=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/3.68k [00:00<?, ?B/s]

california_burned_areas.py:   0%|          | 0.00/5.19k [00:00<?, ?B/s]

512x512.hdf5:   0%|          | 0.00/5.88G [00:00<?, ?B/s]

chabud_test.h5:   0%|          | 0.00/667M [00:00<?, ?B/s]

...

Generating 1 split: 0 examples [00:00, ? examples/s]

Generating 2 split: 0 examples [00:00, ? examples/s]

Generating 3 split: 0 examples [00:00, ? examples/s]

Generating 4 split: 0 examples [00:00, ? examples/s]

Generating chabud split: 0 examples [00:00, ? examples/s]

### На основі аналізу датасету запроповані рішення для перетворення датасету
 - Create an class ratio graphic in output mask

Бінарність класів спрощує Segmentation loss в просту бінарну кроссентропію з логітами.
 Використаємо взваження класів відносно їх частоти в зображенні всередині  лосс функції за допомогою class_weights.
Додаємо до BCE лосс-функції додатково Dice loss, що є диферренційованим F1 score

 - Mean brightness, distribution per-channel of images

Додамо першим шаром батч нормалізацію,

**Створимо синтез фіч**:
1.  порахувавши індекс щільності рослинності (NDVI): гіперболічну дистанцію між яскравістю пікселів Червоного (B04), Інфрачервоного (B08) каналів першим каналом,
2. А також для Довгих інфрачервоних DWDM каналів B12 і B11.

3. Останнім каналом візьмемо B09, для відокремлення вологого грунту
від спаленої землі і водних об'єктів.

Присутність великого набору семплів з пустими масками (class 0) підвищує захищеність моделі

Для зменшення шуму окремо згладимо вхідне зображення з Гауссовим блюром

 - how **correlated** are adjacent 2d tensors in pre-fire, post-fire, could dataset be ordered by **time** if needed?

Так як не присутня залежність між зображеннями в батчі, регуляризація виконується для кожної пари вхідне зображення --- маска:

1. До вхідного зображення додається невелика кількість шуму

2. З RandomJigsaw синхронно розрізаємо зображення та маску на квадрати NxN і перемішуємо їх

3. Під час тренування для одночасно маски і зображення робимо афінне перетворення, відображаємо вертикально та горизонтально

Ціллю є підвищення інваріантності по переміщенню і обертанню, подальше використання dropout для зменшення глобальних глобальних активацій нейронів, і більшого фокусу моделі на локальних фічах, інваріантно від топографії ландшафту і присутніх на ньому географічних ознак.

## Preprocessing functions



- Hyperbolic difference

Відображає нелінійний ріст залежності, накладає вагу для збільшення контрасту між схожими значеннями і аутлаєрами, додає біас.

- Canny (не використовується)

Детектор країв, рахує контури градієнту на зображенні.

In [7]:
def compute_difference(pre_fire, post_fire):
    return post_fire - pre_fire

def hyperbolic_diff(img_x, img_y, scale = 0.5, bias = 0):
    if bias == -1e-6:
        bias = 0
    return scale * (img_x - img_y) / (img_x + img_y + bias + 1e-6)


def canny(image, lo=0.1, hi=0.2):
        return filters.canny(image, low_threshold=lo, high_threshold=hi)

def clip_values(image, min_val=0.0, max_val=1.0):
    return torch.clamp(image, min=min_val, max=max_val)

def unsharp(image):
    return filters.unsharp_maskkernel_size(image, (3, 3), sigma=(0.1, 2.0), border_type='refect')

def rearrange_channels(pre_fire, post_fire):
    # Stack pre-fire and post-fire channels
    return torch.cat([pre_fire, post_fire], dim=1)  # Shape: [Batch, 24, H, W]

Перетворення листків датасету в тензори toTensor замінено на torch.tensor()

- Transform виконується одночасно на вхідному зображенні і масці; стосується лише афінних перетворень без зміни кольорів

- Augmentation - лише для вхідного зображення post_fire - згладжує зображення для ідентифікації грубих фіч, видаляє частину інформації для підвищення робастності на реальному тестуванні

- Important to denoise the data,
 using gaussian blur pyramids

In [8]:
mozaic = K.RandomJigsaw((2, 2), keepdim=True, p=0.25)
# normalize = transforms.Normalize(mean=[0.485]*1, std=[0.229]*1)

gaussian_blur = transforms.GaussianBlur(kernel_size=(3, 3), sigma=(0.1, 2.0))
adjust_contrast = transforms.ColorJitter(contrast=1.5)

totensor = transforms.Compose([
            transforms.Lambda(np.array), # List converts to numpy array
            transforms.ToTensor(),
        ])

transform = transforms.Compose([
    #transforms.Lambda(np.array), # List converts to numpy array
    # mozaic,
    transforms.RandomAffine(degrees=15., translate=(0.3, 0.3), interpolation=PIL.Image.BILINEAR),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    #transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    #filters.Laplacian(kernel_size=(3, 3), border_type='reflect'),
])

augmentation = transforms.Compose([
    # normalize,
    # transforms.GaussianBlur(kernel_size=(3, 3), sigma=(0.1, 2.0)),
    # filters.Laplacian(kernel_size=(3, 3), border_type='reflect'),
    #K.RandomGaussianNoise(std=0.2,  p=0.5),
    transforms.GaussianNoise(sigma=0.2),
    transforms.GaussianBlur(kernel_size=(3, 3), sigma=(0.1, 2.0)),
])



/usr/local/lib/python3.10/dist-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


- Define scripts to convert image to 3 channel image
- Define clipping of class values
-  Normalization
- Showing before/after
- Resizing of images, important for  (Optional)

## Three channels
- (Natural Density of Vegetation, Water vapor intensity, medium-long infrared ratio)

- Розділяємо батч на окремі зображення (3, 512, 512) і маску (1, 512, 512)

Видалені канали:

- B01 - рівень аерозолю

- B02, 03 - зелений і синій, видима неозброєним оком рослинність і водойми

- B05-07 - Класифікація рослинних культур

- B10 - Тип хмар

- Всі інші канали крім B04, B08, B09, B11, B12 дропаємо для зменшення зайвого об'єму інформації поданого на вхід, і використання претренованих ваг.


In [9]:
# Define a custom PyTorch Dataset to apply transformations
class CaBuArPyTorchDataset(datasets.GeneratorBasedBuilder):
    def __init__(self, hf_dataset, transform=None, augment=None):
    # Commented - Alternative def using torchgeo parent class
    #def __init__(self, root, transform=None, download = False):
        """
        Args:
            hf_dataset: Hugging Face dataset object
            transform: Optional transform to be applied on a sample
            download: if True, download dataset and store it in the root directory
        """
        # super().__init__(root=root, transforms=transform, download=download)
        self.hf_dataset = hf_dataset
        self.transform = transform
        self.augment = augment


    def calculate_ndvi(self, post_fire):
        # Assuming NIR is channel 8 and Red is channel 4 (0-indexed)
        # pre_nir = pre_fire[:, 7, :, :].unsqueeze(0)
        # pre_red = pre_fire[:, 3, :, :].unsqueeze(0)
        # post_nir = post_fire[:, 7, :, :].unsqueeze(0)
        # post_red = post_fire[:, 3, :, :].unsqueeze(0)
        # pre_nir = pre_fire[0, 7, :, :]
        # pre_red = pre_fire[0, 3, :, :]
        post_nir = post_fire[7, :, :]
        post_red = post_fire[3, :, :]
        # Compute hyperbolic difference of channels
        # ndvi_pre = hyperbolic_diff(pre_nir, pre_red, scale=1.)
        ndvi_post = hyperbolic_diff(post_nir, post_red, scale=1.)

        # Return a mean square distance between old and new
        #return torch.square(ndvi_post - ndvi_pre)
        return ndvi_post


    def fire_boundary(self, post_fire, lo=0.1, hi=0.2):
        # Extract low frequency bands:
        # band_11 = post_fire[:, 10, :, :].unsqueeze(0)
        # band_12 = post_fire[:, 11, :, :].unsqueeze(0)
        band_11 = post_fire[10, :, :]
        band_12 = post_fire[11, :, :]

        return hyperbolic_diff(band_11, band_12, 8.5, 0.25)  # Shape: [1, H, W]

    def __len__(self):
        return len(self.hf_dataset)

    def __getitem__(self, idx):
        sample = self.hf_dataset[idx]
        post_fire = sample['post_fire']  # Shape: (512, 512, 12)
        mask = sample['mask']            # Shape: (512, 512, 1)
        #print(mask.shape)
        #pre_fire = sample.get('pre_fire', np.zeros_like(post_fire))

        # Convert list files into tensors
        #pre_fire = totensor(pre_fire)
        post_fire = torch.tensor(post_fire, dtype=torch.float).permute(2, 0, 1)

        print(f"Post fire {post_fire.shape}")
        mask = torch.tensor(mask, dtype=torch.float).permute(2, 0, 1)
        print(f"Mask {mask.shape}")
        # Calculate NDVI - natural vegetation index
        ndvi = self.calculate_ndvi(post_fire)
        print(f"NDVI {ndvi.shape}")
        # Compute fire boundary using sentinel script
        fire_boundary = self.fire_boundary(post_fire)
        #print(f"Boundary {fire_boundary.shape}")
        # Extract water vapor band 09
        # water_vapor = post_fire[:, 8, :, :].unsqueeze(0)
        water_vapor = post_fire[8, :, :]
        #print(f"Vapor {water_vapor.shape}")
        # Augment layers: edge detection, saturation etc.
        if self.augment:
            ndvi = self.augment(ndvi)
            water_vapor = self.augment(water_vapor)
            fire_boundary = self.augment(fire_boundary)
        #print(f"Augmented {ndvi.shape}")
        # Concatenate pre-fire, post-fire, NDVI, difference
        combined_input = torch.stack([ndvi.squeeze(), water_vapor.squeeze(), fire_boundary.squeeze(), mask.squeeze()], dim=0)  # 4 channels]
        #print(combined_input.shape)
        # Transform all layers uniformly, to maintain ground truth on validation
       # combined_input = combined_input.unsqueeze(0)  # Add batch dimension for transforms
        if self.transform:
            combined_input = self.transform(combined_input)

        # Move tensors to device
        # combined_input = combined_input.to(device)
        # pre_fire = combined_input[:, :13, :, :]
        # post_fire = combined_input[:, 13:25, :, :]
        # mask = combined_input[: 25, :, :]
        # ndvi_pre = combined_input[:, 26, :, :]
        # ndvi_post = combined_input[:, 27, :, :]
        # diff_image = combined_input[:, 27:, :, :]




        # Prepare the X_input and y
        return combined_input[:-1], combined_input[-1]

Обгортка для передачі даних на CUDA

In [10]:
class WrappedDataLoader:
    def __init__(self, loader, func):
        self.loader = loader
        self.func = func

    def __len__(self):
        return len(self.loader)

    def __iter__(self):
        for batch in iter(self.loader):
            yield self.func(*batch)

Train, validation і test лоадери

In [11]:
# Instantiate the custom dataset
train_dataset = CaBuArPyTorchDataset(
    hf_dataset=train_data,
    transform=transform,
    augment=augmentation,
)
valid_dataset = CaBuArPyTorchDataset(
    hf_dataset=valid_data,
    transform=transform,
    augment=augmentation,
)
test_dataset = CaBuArPyTorchDataset(
    hf_dataset=test_data,
)

tr_loader = WrappedDataLoader(DataLoader(
    train_dataset,
    batch_size=_BATCH_SIZE,       # Adjust batch size as needed
    shuffle=False,        # Shuffle for training
    num_workers=_NUM_WORKERS,       # Number of subprocesses for data loading
    pin_memory=True      # Speed up transfer to GPU
), to_device)
vd_loadr = WrappedDataLoader(DataLoader(
    valid_dataset,
    batch_size=_BATCH_SIZE,       # Adjust batch size as needed
    shuffle=True,        # Shuffle for training
    num_workers=_NUM_WORKERS,       # Number of subprocesses for data loading
    pin_memory=True      # Speed up transfer to GPU
), to_device)
test_loadr = WrappedDataLoader(DataLoader(
    test_dataset,
    batch_size=_BATCH_SIZE,       # Adjust batch size as needed
    shuffle=False,        # Shuffle for training
    num_workers=_NUM_WORKERS,       # Number of subprocesses for data loading
    pin_memory=True      # Speed up transfer to GPU
), to_device)

Перевірка на доступність до завантаження файлів датасету

In [ ]:
%%time
# Example: Iterate through the DataLoader
for batch_idx, batch in enumerate(tr_loader):
    X, y = batch

    # Now you can pass `post_fire`, `pre_fire`, and `mask` to your model
    # Example:
    # outputs = model(post_fire, pre_fire)
    # loss = criterion(outputs, mask)

    # For demonstration, we'll just print the batch shapes
    print(f"Batch {batch_idx}:")
    print(f"  post_fire shape: {X.shape}")
    # if pre_fire is not None:
    #     print(f"  pre_fire shape: {pre_fire.shape}")
    print(f"  mask shape: {y.shape}")

    # Break after first batch for demonstration
    if batch_idx == 0:
        break

### Відображення результатів препроцессингу

- Виділимо 5 випадкових зображень з train і test спліту

- Train додає трансформації до зображення

- Test показує оригінал без перетворень

In [13]:
idx, processed_img = next(enumerate(tr_loader))

Post fire torch.Size([12, 512, 512])
Post fire torch.Size([12, 512, 512])
Mask torch.Size([1, 512, 512])
NDVI torch.Size([512, 512])
Mask torch.Size([1, 512, 512])
NDVI torch.Size([512, 512])


IndexError: Caught IndexError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/worker.py", line 351, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/fetch.py", line 52, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/fetch.py", line 52, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "<ipython-input-9-624207f0fae7>", line 75, in __getitem__
    ndvi = self.augment(ndvi)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1736, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1747, in _call_impl
    return forward_call(*arg

In [ ]:
X_train, y = processed_img

In [ ]:

# Let's plot some images

fig, axes = plt.subplots(5, 2, figsize=(10,15))
ch = 0
fig.suptitle(f"train  channel {ch}", fontsize=20)
for i in range(5):
    axes[i%5,0].imshow(X_train[i,ch].view(512,512,1).numpy())
    axes[i%5,1].imshow(y[i].view(512,512,).numpy())
plt.show()
    #plt.title(str(y_train[i].item()))
#j = 0
# for cls in clss_counts.index:
#     cls_indices = np.where(y_train.numpy() == cls)[0]
#     if len(cls_indices) > 0:
#         plt.subplot(2, len(clss_counts), j + 1)
#         plt.axis('off')
#         plt.imshow(X_train[cls_indices[0], :, :, :].numpy().reshape(28, 28), cmap="gray_r")
#         plt.title(str(cls))
#         j += 1

In [ ]:
idx, test_img = next(enumerate(test_loadr))

In [ ]:
X_train, y = test_img

In [ ]:

# Let's plot some images
fig, axes = plt.subplots(5, 2, figsize=(10,15))
ch = 2
fig.suptitle(f"test channel {ch}", fontsize=20)
for i in range(5):
    axes[i,0].imshow(X_train[i+5,ch].view(512,512,1).numpy())
    axes[i,1].imshow(y[i+5].view(512,512,).numpy())
plt.show()
    #plt.title(str(y_train[i].item()))
#j = 0
# for cls in clss_counts.index:
#     cls_indices = np.where(y_train.numpy() == cls)[0]
#     if len(cls_indices) > 0:
#         plt.subplot(2, len(clss_counts), j + 1)
#         plt.axis('off')
#         plt.imshow(X_train[cls_indices[0], :, :, :].numpy().reshape(28, 28), cmap="gray_r")
#         plt.title(str(cls))
#         j += 1

## Використовуємо за Loss Combined BCE + F1 (Dice loss)

## Метрики - попіксельно F1/AuC та Extended IoU

Покращений IoU враховує відстань між масками і ground truth навіть за нульового перекриття, уникає плато нульового градієнту.

### Додамо ваги класів в лосс для урахування дизбалансу.

In [ ]:
### Class imbalance (sometimes a few pixels against a backdrop of a single class)
### Should be solved by applying weight for error on each class by its size

def calculate_class_weights(y):
    unique_classes, class_counts = np.unique(y, return_counts=True)
    total_samples = len(y)
    return list(map(lambda x: total_samples / len(unique_classes) / class_counts[x], unique_classes))
class_weights = torch.tensor(calculate_class_weights(y_train), dtype=torch.float, device=device)

criterion = nn.CrossEntropyLoss(class_weights)

# AlexNet + Clusterization

In [ ]:
from models import AlexNet
from models import KMeansClustering
model = AlexNet()
print(model)
class SemanticSegmentationNetwork(nn.Module):
    def __init__(self, num_clusters=6, input_channels=9, num_classes=6):
        super(SemanticSegmentationNetwork, self).__init__()
        self.feature_extractor = AlexNet(input_channels=input_channels)
        self.segmentation_head = KMeansClustering(num_clusters=num_clusters, num_features=256)

    def forward(self, pre_fire, post_fire, edge_image):
        combined_input = torch.cat((pre_fire, post_fire, edge_image), dim=1)
        print(f"convolution in shape: {combined_input.shape}")
        features = self.feature_extractor(combined_input)
        print(f"convolution out shape: {features.shape}")
        segmentation_mask = self.segmentation_head(features)
        return segmentation_mask

In [ ]:

#from models.semantic_segmentation import SemanticSegmentationNetwork

# Create dummy data
batch_size, height, width = 1, 224, 224
pre_fire = torch.randn(batch_size, 3, height, width)
post_fire = torch.randn(batch_size, 3, height, width)
edge_image = torch.randn(batch_size, 3, height, width)

# Initialize the network
model = SemanticSegmentationNetwork(num_clusters=6, input_channels=9)

# Forward pass
segmentation_mask = model(pre_fire, post_fire, edge_image)

# Print the output shape
print("Segmentation Mask Shape:", segmentation_mask.shape)

# Пайплайн тренування і валідації

In [ ]:
# (Optional) Collect accuracy vector after training epochs
# Here, it's assumed you have a training loop where you collect accuracy

# Example training loop structure
def train_model(dataloader, model, criterion, optimizer, num_epochs=10):
    loss_vector = []
    accuracy_vector = []

    for epoch in range(1, num_epochs + 1):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for batch in dataloader:
            post_fire = batch['post_fire']
            mask = batch['mask']
            pre_fire = batch.get('pre_fire')

            # Move data to device
            post_fire = post_fire.to(device)
            mask = mask.to(device)
            if pre_fire is not None:
                pre_fire = pre_fire.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(post_fire, pre_fire) if pre_fire is not None else model(post_fire)
            loss = criterion(outputs, mask)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            # Statistics
            running_loss += loss.item() * post_fire.size(0)
            # Assuming outputs are logits; apply argmax to get predictions
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == mask).sum().item()
            total += mask.numel()

        epoch_loss = running_loss / len(pytorch_dataset)
        epoch_acc = 100. * correct / total
        loss_vector.append(epoch_loss)
        accuracy_vector.append(epoch_acc)

        print(f"Epoch {epoch}/{num_epochs}: Loss={epoch_loss:.4f}, Accuracy={epoch_acc:.2f}%")

    print("Training complete.")
    print("Accuracy vector:", accuracy_vector)
    return loss_vector, accuracy_vector